In [64]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [112]:
df = pd.read_csv('../data/health_data_CA.csv', parse_dates=['VISITDATE'])
print(f'Shape of df: {df.shape}\n')
df.head()

Shape of df: (64229, 12)



,HEALTHOFFICEVISITID,HEALTHMAINSTUDRECID,VISITDATE,VISITTIMEIN,VISITTIMEOUT,ISGUARDIANCONTACTED,VISITREASONDESC,ASSESSMENT,ACTIONS,SCHOOLNUMBER,STUDENTID,DISTRICT_NAME
0,340002,52041,2018-10-19,2018-10-19 10:56:00.000,2018-10-19 11:00:00.000,0,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,173453,CA
1,364646,52041,2019-09-09,2019-09-09 10:03:00.000,2019-09-09 10:15:00.000,1,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,173453,CA
2,324248,137469,2018-03-14,2018-03-14 12:21:24.067,2018-03-14 12:21:24.067,1,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,237352,CA
3,321440,137469,2018-02-15,2018-02-15 11:03:00.000,2018-02-15 11:30:00.000,0,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,237352,CA
4,350705,134330,2019-03-05,2019-03-05 09:11:00.000,2019-03-05 09:11:00.000,1,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,220603,CA


In [113]:
# Info on dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64229 entries, 0 to 64228
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   HEALTHOFFICEVISITID  64229 non-null  int64         
 1   HEALTHMAINSTUDRECID  64229 non-null  int64         
 2   VISITDATE            64229 non-null  datetime64[ns]
 3   VISITTIMEIN          64229 non-null  object        
 4   VISITTIMEOUT         35002 non-null  object        
 5   ISGUARDIANCONTACTED  64229 non-null  int64         
 6   VISITREASONDESC      64172 non-null  object        
 7   ASSESSMENT           15388 non-null  object        
 8   ACTIONS              25187 non-null  object        
 9   SCHOOLNUMBER         64229 non-null  int64         
 10  STUDENTID            64229 non-null  int64         
 11  DISTRICT_NAME        64229 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 5.9+ MB


In [21]:
# Missing values
df.isna().sum()

HEALTHOFFICEVISITID        0
HEALTHMAINSTUDRECID        0
VISITDATE                  0
VISITTIMEIN                0
VISITTIMEOUT           29227
ISGUARDIANCONTACTED        0
VISITREASONDESC           57
ASSESSMENT             48841
ACTIONS                39042
SCHOOLNUMBER               0
STUDENTID                  0
DISTRICT_NAME              0
dtype: int64

- VISITREASONDESC, ASSESSMENT, ACTIONS are descriptive cols wihich doesn't have much info for our case and these are cols which have NaNs

- VISITTIMEOUT: gives the time when student is out from doc, this has almost ~50% NaNs. One thing we may get from this is taking the difference of In and Out time which may give an idea of severeness of the issue and may have an impact on final attendance. This can also be seen wrt 'ISGUARDIANCONTACTED' col.

- If the time diff is more and guardian is contacted then the probability of issue being severe is more and inturn have an impact on attendance.

In [22]:
print(f'Unique reasons/types of reasons mentioned: {df.VISITREASONDESC.unique()}\n')
print(f'Assesment by doc: {df.ASSESSMENT.unique()}\n')
print(f'Action taken: {df.ACTIONS.unique()}')

Unique reasons/types of reasons mentioned: ['Visit reason goes here...' nan]

Assesment by doc: ['Assessment goes here...' nan]

Action taken: ['Action goes here...' nan]


In [23]:
# convert object to datetime
df['VISITDATE'] = df['VISITDATE'].apply(pd.to_datetime)
df['VISITTIMEIN'] = df['VISITTIMEIN'].apply(pd.to_datetime)
df['VISITTIMEOUT'] = df['VISITTIMEOUT'].apply(pd.to_datetime)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64229 entries, 0 to 64228
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   HEALTHOFFICEVISITID  64229 non-null  int64         
 1   HEALTHMAINSTUDRECID  64229 non-null  int64         
 2   VISITDATE            64229 non-null  datetime64[ns]
 3   VISITTIMEIN          64229 non-null  datetime64[ns]
 4   VISITTIMEOUT         35002 non-null  datetime64[ns]
 5   ISGUARDIANCONTACTED  64229 non-null  int64         
 6   VISITREASONDESC      64172 non-null  object        
 7   ASSESSMENT           15388 non-null  object        
 8   ACTIONS              25187 non-null  object        
 9   SCHOOLNUMBER         64229 non-null  int64         
 10  STUDENTID            64229 non-null  int64         
 11  DISTRICT_NAME        64229 non-null  object        
dtypes: datetime64[ns](3), int64(5), object(4)
memory usage: 5.9+ MB


In [29]:
# VISITTIMEOUT contains ~50% NaNs, let's drop them and create a new df for our understanding/exploration
df1 = df.dropna()
df1 = df1.drop_duplicates() # No duplicates actually
print(f'Shape: {df1.shape}\nAny missing values: \n{df1.isnull().sum()}')

Shape: (11544, 12)
Any missing values: 
HEALTHOFFICEVISITID    0
HEALTHMAINSTUDRECID    0
VISITDATE              0
VISITTIMEIN            0
VISITTIMEOUT           0
ISGUARDIANCONTACTED    0
VISITREASONDESC        0
ASSESSMENT             0
ACTIONS                0
SCHOOLNUMBER           0
STUDENTID              0
DISTRICT_NAME          0
dtype: int64


In [49]:
# extract time from VISITTIMEIN and VISITTIMEOUT cols
df1['IN_date'] = pd.to_datetime(df1['VISITTIMEIN']).dt.date
df1['IN_time'] = pd.to_datetime(df1['VISITTIMEIN']).dt.time
df1['OUT_date'] = pd.to_datetime(df1['VISITTIMEOUT']).dt.date
df1['OUT_time'] = pd.to_datetime(df1['VISITTIMEOUT']).dt.time
df1.head()

,HEALTHOFFICEVISITID,HEALTHMAINSTUDRECID,VISITDATE,VISITTIMEIN,VISITTIMEOUT,ISGUARDIANCONTACTED,VISITREASONDESC,ASSESSMENT,ACTIONS,SCHOOLNUMBER,STUDENTID,DISTRICT_NAME,IN_date,IN_time,OUT_date,OUT_time
0,340002,52041,2018-10-19,2018-10-19 10:56:00.000,2018-10-19 11:00:00.000,0,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,173453,CA,2018-10-19,10:56:00,2018-10-19,11:00:00
1,364646,52041,2019-09-09,2019-09-09 10:03:00.000,2019-09-09 10:15:00.000,1,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,173453,CA,2019-09-09,10:03:00,2019-09-09,10:15:00
2,324248,137469,2018-03-14,2018-03-14 12:21:24.067,2018-03-14 12:21:24.067,1,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,237352,CA,2018-03-14,12:21:24.067000,2018-03-14,12:21:24.067000
3,321440,137469,2018-02-15,2018-02-15 11:03:00.000,2018-02-15 11:30:00.000,0,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,237352,CA,2018-02-15,11:03:00,2018-02-15,11:30:00
4,350705,134330,2019-03-05,2019-03-05 09:11:00.000,2019-03-05 09:11:00.000,1,Visit reason goes here...,Assessment goes here...,Action goes here...,1936624,220603,CA,2019-03-05,09:11:00,2019-03-05,09:11:00


In [76]:
df1['VISITTIMEIN'] - df1['VISITTIMEOUT']

0       -1 days +23:56:00
1       -1 days +23:48:00
2         0 days 00:00:00
3       -1 days +23:33:00
4         0 days 00:00:00
               ...       
64171   -1 days +23:48:00
64172     0 days 00:00:00
64173   -1 days +11:57:00
64180   -1 days +23:52:00
64181     0 days 00:00:00
Length: 11544, dtype: timedelta64[ns]

In [50]:
# sample check for diff btw In and Out date, by some wrong entry
if np.subtract((pd.DatetimeIndex(df1['IN_date']).day), (pd.DatetimeIndex(df1['OUT_date']).day)).all() == 0:
    print('All Good')
else:
    print('Check idx', np.where(np.subtract((pd.DatetimeIndex(df1['IN_date']).day), (pd.DatetimeIndex(df1['OUT_date']).day)))).all()

All Good


So we can eliminate the day col, and look for difference in time in and out.

In [51]:
df1.columns

Index(['HEALTHOFFICEVISITID', 'HEALTHMAINSTUDRECID', 'VISITDATE',
       'VISITTIMEIN', 'VISITTIMEOUT', 'ISGUARDIANCONTACTED', 'VISITREASONDESC',
       'ASSESSMENT', 'ACTIONS', 'SCHOOLNUMBER', 'STUDENTID', 'DISTRICT_NAME',
       'IN_date', 'IN_time', 'OUT_date', 'OUT_time'],
      dtype='object')

In [52]:
# drop the cols which aren't required for our ease
df2 = df1.drop([
    'VISITTIMEIN', 'VISITTIMEOUT', 'VISITREASONDESC', 'ASSESSMENT', 'ACTIONS', 'DISTRICT_NAME', 'IN_date', 'OUT_date'
    ], axis=1)
print(f'Shape: {df2.shape}\n, Any NaNs: \n{df2.isna().sum()}')
df2.head()

Shape: (11544, 8)
, Any NaNs: 
HEALTHOFFICEVISITID    0
HEALTHMAINSTUDRECID    0
VISITDATE              0
ISGUARDIANCONTACTED    0
SCHOOLNUMBER           0
STUDENTID              0
IN_time                0
OUT_time               0
dtype: int64


,HEALTHOFFICEVISITID,HEALTHMAINSTUDRECID,VISITDATE,ISGUARDIANCONTACTED,SCHOOLNUMBER,STUDENTID,IN_time,OUT_time
0,340002,52041,2018-10-19,0,1936624,173453,10:56:00,11:00:00
1,364646,52041,2019-09-09,1,1936624,173453,10:03:00,10:15:00
2,324248,137469,2018-03-14,1,1936624,237352,12:21:24.067000,12:21:24.067000
3,321440,137469,2018-02-15,0,1936624,237352,11:03:00,11:30:00
4,350705,134330,2019-03-05,1,1936624,220603,09:11:00,09:11:00


In [85]:
(df2.IN_time != df2.OUT_time).value_counts()

True     9897
False    1647
dtype: int64

- There are 1647 values where ther is time difference, lets see if these have guardian contacted = 1

In [96]:
df3 = df2[(df2.IN_time != df2.OUT_time) & (df2.ISGUARDIANCONTACTED == 1)]
print('Shape:', df3.shape)
df3.head()

Shape: (6095, 8)


,HEALTHOFFICEVISITID,HEALTHMAINSTUDRECID,VISITDATE,ISGUARDIANCONTACTED,SCHOOLNUMBER,STUDENTID,IN_time,OUT_time
1,364646,52041,2019-09-09,1,1936624,173453,10:03:00,10:15:00
22,366760,147219,2019-09-23,1,1995844,296810,11:50:00,12:05:00
67,361654,143700,2019-08-16,1,1995844,301625,10:10:00,10:13:00
72,353943,143879,2019-04-12,1,1936624,306697,08:26:00,08:28:00
90,365823,146986,2019-09-17,1,1995406,300675,09:21:00,09:39:00


- This df3 may contribute to attendance: As there is diff in In and Out time and reported to guardian

In [101]:
(pd.DatetimeIndex(df3['VISITDATE']).year).value_counts()

2019    3174
2018    2037
2020     884
Name: VISITDATE, dtype: int64